In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from keras.models import load_model

2025-02-11 17:33:02.179536: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-11 17:33:02.179807: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-11 17:33:02.442545: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
csv = '/kaggle/input/disc-golf-disc-flight-numbers-and-dimensions/disc-data.csv'
df = pd.read_csv(csv)
df['MOLD'] = df['MOLD'].str.replace('\n', '')
print(df.describe())

             SPEED        GLIDE         TURN         FADE    STABILITY  \
count  1175.000000  1175.000000  1175.000000  1175.000000  1175.000000   
mean      6.985532     4.291064    -0.839574     2.014894     1.173617   
std       3.676466     1.110181     1.122030     1.133728     1.890662   
min       1.000000     0.000000    -5.000000     0.000000    -4.000000   
25%       4.000000     4.000000    -1.000000     1.000000     0.000000   
50%       7.000000     4.500000    -0.500000     2.000000     1.000000   
75%      10.000000     5.000000     0.000000     3.000000     2.500000   
max      14.500000     7.000000     2.000000     6.000000     7.000000   

       DIAMETER (cm)  HEIGHT (cm)  RIM DEPTH (cm)  RIM WIDTH (cm)  \
count    1175.000000  1175.000000     1175.000000     1175.000000   
mean       21.297191     1.811830        1.259574        1.647574   
std         0.330815     0.229446        0.152016        0.480834   
min        21.000000     1.300000        1.100000        

In [3]:
print(df.shape)
df[['BEAD']] = df[['BEAD']].replace('No',0)
df[['BEAD']] = df[['BEAD']].replace('Yes',1)
Y = df[["SPEED", "GLIDE", "TURN", "FADE"]]
X = df.loc[:,~df.columns.isin(["SPEED", "GLIDE", "TURN", "FADE"])]
print(X.shape)
print(Y.shape)
print(X)

(1175, 15)
(1175, 11)
(1175, 4)
            MOLD        DISC TYPE  STABILITY  DIAMETER (cm)  HEIGHT (cm)  \
0      Armadillo  Putt & Approach        1.0           21.3          1.9   
1         Beetle  Putt & Approach       -1.0           21.9          2.2   
2           Berg  Putt & Approach        2.0           21.1          2.0   
3         Birdie  Putt & Approach        0.0           21.2          2.4   
4          Claws  Putt & Approach        0.5           21.2          2.0   
...          ...              ...        ...            ...          ...   
1170    Delirium  Distance Driver        2.5           21.1          1.8   
1171   Dimension  Distance Driver        3.0           21.0          1.8   
1172      Excite  Distance Driver        0.0           21.1          2.1   
1173       Limit  Distance Driver        4.0           21.1          1.8   
1174  Relativity  Distance Driver       -1.5           21.1          1.9   

      RIM DEPTH (cm)  RIM WIDTH (cm)  INSIDE RIM DIAMET

/tmp/ipykernel_18/3062733839.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[['BEAD']] = df[['BEAD']].replace('Yes',1)


In [4]:
# Calculate the number of rows for each split
total_rows = df.shape[0]
train_rows = int(0.8 * total_rows)
validation_rows = int(0.1 * total_rows)
test_rows = total_rows - train_rows - validation_rows

# Generate random indices for splitting
indices = np.arange(total_rows)
np.random.shuffle(indices)

# Split the indices
train_indices = indices[:train_rows]
validation_indices = indices[train_rows:train_rows + validation_rows]
test_indices = indices[train_rows + validation_rows:]

# Create new arrays based on the indices
X_train = X.loc[train_indices, :]
Y_train = Y.loc[train_indices, :]
X_validation = X.loc[validation_indices, :]
Y_validation = Y.loc[validation_indices, :]
X_test = X.loc[test_indices, :]
Y_test = Y.loc[test_indices]

# Verify the shapes of the new arrays
print("Shape of X_train:", X_train.shape)
print("Shape of Y_train:", Y_train.shape)
print("Shape of X_validation:", X_validation.shape)
print("Shape of Y_validation:", Y_validation.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of Y_test:", Y_test.shape)

X_train = X_train.loc[:,~X_train.columns.isin(["MOLD", "DISC TYPE"])]
X_validation = X_validation.loc[:,~X_validation.columns.isin(["MOLD", "DISC TYPE"])]
X_test = X_test.loc[:,~X_test.columns.isin(["MOLD", "DISC TYPE"])]

X_train = np.array(X_train)
Y_train = np.array(Y_train)
X_validation = np.array(X_validation)
Y_validation = np.array(Y_validation)
X_test = np.array(X_test)
Y_test = np.array(Y_test)

Shape of X_train: (940, 11)
Shape of Y_train: (940, 4)
Shape of X_validation: (117, 11)
Shape of Y_validation: (117, 4)
Shape of X_test: (118, 11)
Shape of Y_test: (118, 4)


In [5]:
X_train = np.array(X_train)
X_validation = np.array(X_validation)
X_test = np.array(X_test)

print(f"Diameter Max, Min pre normalization: {np.max(X_train[0:,1]):0.2f}, {np.min(X_train[:,1]):0.2f}")
print(f"Inside Rim Diameter Max, Min pre normalization: {np.max(X_train[:,5]):0.2f}, {np.min(X_train[:,5]):0.2f}")
print(f"Rim Depth/Diameter Ratio Max, Min pre normalization: {np.max(X_train[:,6]):0.2f}, {np.min(X_train[:,6]):0.2f}")
print(f"Rim Configuration Max, Min pre normalization: {np.max(X_train[:,7]):0.2f}, {np.min(X_train[:,7]):0.2f}")

norm_l = tf.keras.layers.Normalization(axis=-1)
norm_2 = tf.keras.layers.Normalization(axis=-1)
norm_3 = tf.keras.layers.Normalization(axis=-1)
norm_l.adapt(X_train)  # learns mean, variance
norm_2.adapt(X_validation)
norm_3.adapt(X_test)
X_train_n = norm_l(X_train)
X_validation_n = norm_2(X_validation)
X_test_n = norm_3(X_test)

print(f"Diameter Max, Min post normalization: {np.max(X_train_n[0:,1]):0.2f}, {np.min(X_train_n[:,1]):0.2f}")
print(f"Inside Rim Diameter Max, Min post normalization: {np.max(X_train_n[:,5]):0.2f}, {np.min(X_train_n[:,5]):0.2f}")
print(f"Rim Depth/Diameter Ratio Max, Min post normalization: {np.max(X_train_n[:,6]):0.2f}, {np.min(X_train_n[:,6]):0.2f}")
print(f"Rim Configuration Max, Min post normalization: {np.max(X_train_n[:,7]):0.2f}, {np.min(X_train_n[:,7]):0.2f}")


Diameter Max, Min pre normalization: 24.20, 21.00
Inside Rim Diameter Max, Min pre normalization: 22.80, 15.90
Rim Depth/Diameter Ratio Max, Min pre normalization: 9.70, 5.00
Rim Configuration Max, Min pre normalization: 98.25, 26.00
Diameter Max, Min post normalization: 9.04, -0.91
Inside Rim Diameter Max, Min post normalization: 4.36, -1.90
Rim Depth/Diameter Ratio Max, Min post normalization: 5.30, -1.30
Rim Configuration Max, Min post normalization: 3.95, -1.01


In [6]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model = Sequential(
    [
        Dense(10, activation='relu', name = 'layer1'),
        Dropout(0.2),
        Dense(9, activation='relu', name = 'layer2'),
        Dropout(0.4),
        Dense(4, activation='relu', name = 'layer3')

     ]
)

In [7]:
model.compile(optimizer = tf.keras.optimizers.Adadelta(learning_rate=0.2),
              loss = tf.keras.losses.MeanSquaredError(),
              metrics = ['accuracy'])
model.fit(
    X_train_n,Y_train, 
    batch_size = 32,
    epochs=200,
    verbose = 2,
    validation_data = (X_validation_n, Y_validation)
)

score = model.evaluate(X_test_n, Y_test, verbose = 0)
print('Test loss: ', score[0])
print('Test accuracy: ', score[1])

Epoch 1/200
30/30 - 1s - 40ms/step - accuracy: 0.2117 - loss: 21.5761 - val_accuracy: 0.1795 - val_loss: 21.2698
Epoch 2/200
30/30 - 0s - 3ms/step - accuracy: 0.1957 - loss: 21.4606 - val_accuracy: 0.1709 - val_loss: 21.0356
Epoch 3/200
30/30 - 0s - 3ms/step - accuracy: 0.2085 - loss: 21.1960 - val_accuracy: 0.1795 - val_loss: 20.7548
Epoch 4/200
30/30 - 0s - 2ms/step - accuracy: 0.2330 - loss: 20.8325 - val_accuracy: 0.1880 - val_loss: 20.4216
Epoch 5/200
30/30 - 0s - 3ms/step - accuracy: 0.2138 - loss: 20.6351 - val_accuracy: 0.1966 - val_loss: 20.0575
Epoch 6/200
30/30 - 0s - 3ms/step - accuracy: 0.2096 - loss: 20.2217 - val_accuracy: 0.1966 - val_loss: 19.6512
Epoch 7/200
30/30 - 0s - 3ms/step - accuracy: 0.2287 - loss: 19.8837 - val_accuracy: 0.2137 - val_loss: 19.1548
Epoch 8/200
30/30 - 0s - 3ms/step - accuracy: 0.2340 - loss: 19.4662 - val_accuracy: 0.2051 - val_loss: 18.5318
Epoch 9/200
30/30 - 0s - 3ms/step - accuracy: 0.2309 - loss: 18.8728 - val_accuracy: 0.2051 - val_loss:

In [8]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ layer1 (Dense)                  │ (None, 10)             │           100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 10)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer2 (Dense)                  │ (None, 9)              │            99 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 9)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer3 (Dense)                  │ (None, 4)              │            40 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 719 (2.81 KB)

 Trainable params: 239 (956.00 B)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 480 (1.88 KB)

In [9]:

model.save('my_model.h5')  # creates a HDF5 file 'my_model.h5'
del model  # deletes the existing model

# returns a compiled model
# identical to the previous one
model = load_model('my_model.h5')

In [10]:
W1, b1 = model.get_layer("layer1").get_weights()
W2, b2 = model.get_layer("layer2").get_weights()
print("W1:\n", W1, "\nb1:", b1)
print("W2:\n", W2, "\nb2:", b2)

W1:
 [[ 0.5722509   0.21062629 -0.14830129 -0.06349944 -0.07737809 -0.03526865
  -0.20675553 -0.35163248  0.24489073 -0.5748606 ]
 [ 0.40988395 -0.2682261  -0.07262499 -0.17292939 -0.34986433 -0.05625328
  -0.13786472 -0.08786851  0.07191488  0.52100766]
 [-0.15761061  0.32755223 -0.43850517 -0.29598442 -0.3392508  -0.00216763
   0.1567056   0.20043333  0.2434838  -0.32676238]
 [-0.39909622  0.39764684 -0.35920602 -0.26474828 -0.29163304 -0.07763721
   0.27019763  0.16565706 -0.17694648  0.13986291]
 [ 0.03531423  0.74487877  0.2015494   0.6629659  -0.12496899  0.33512855
  -0.1695465   0.4186064   0.08099569 -0.29683924]
 [ 0.18183357 -0.47666526  0.10940319  0.29490778 -0.22356069 -0.23126282
   0.20919418  0.02306103 -0.1851689  -0.36704156]
 [ 0.4016356  -0.3332086   0.45056343  0.26819068 -0.49184346  0.25415713
   0.1898388  -0.32880074  0.06697115 -0.08820713]
 [-0.54761577 -0.04417923 -0.50910115  0.16255496 -0.11057612  0.00952765
   0.09760952 -0.6346595  -0.2525094   0.36409

In [11]:
def predict_with_model(df, row_name, model):
    # Assuming df is your pandas DataFrame
    # Extract the specified row
    selected_row = np.array(df[df['MOLD'] == row_name])
    # Extract columns 6-14
    features = selected_row[0][6:15]

  # Create a Keras normalization layer
    normalization_layer = tf.keras.layers.Normalization(axis=-1)
    normalization_layer.adapt(X_test)  # Compute mean and variance

    # Normalize the features
    normalized_features = normalization_layer(features)
    print(normalized_features)
    # Predict using the model
    predicted_values = np.round(model.predict(normalized_features))

    # Print results
    print(f"Row Name: {row_name}")
    print(f"Predicted Values: {predicted_values}")
    print(f"Actual Values: {selected_row[0][2:6]}")  # Assuming 'target' is the actual value column
    

In [12]:
predict_with_model(df,'Force',model)

tf.Tensor(
[[ 0.9466042  -0.6030015  -0.40230379 -0.37754315  1.3533851  -1.3349031
  -0.25482324 -1.0005109  -0.32063022]], shape=(1, 9), dtype=float32)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Row Name: Force
Predicted Values: [[10.  5.  0.  3.]]
Actual Values: [12.0 5.0 0.0 3.0]
